In [2]:
#从这里开始
#这里两个部分都是一样的 从同一张表格出发

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

all_data=pd.read_csv('D:/new_date.csv',low_memory=False)
print(all_data.shape)

#这里把数据集打乱 防止人都出现在一起
all_data = all_data.sample(frac=1.0)
all_data = all_data.reset_index()
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
print(missing_data.head)

#删除一个样本 也许会漏掉一个user_id 这里我的态度很坚决 漏就漏了 我赋值0.5
#因为假数据也是猜 为什么不猜一手大的？

all_data.drop(all_data[all_data['customer_province']=='柔佛'].index,inplace=True)
print(all_data.shape)

all_data['is_customer_rate'].fillna(0,inplace=True)
all_data['customer_gender'].fillna(0,inplace=True)
all_data['is_member_actived'].fillna(0,inplace=True)

#member_status 有点奇怪 只有1  所以填充的时候要附上一个2 
#all_data['member_status'].plot()
#子订单应付总金额不能填充0 请注意 实在是太多了所以删除掉

all_data['member_status'].fillna(2,inplace=True)
all_data.drop(columns=['order_detail_amount'],inplace=True)
all_data.drop(columns=['customer_city','customer_province'],inplace=True)
#把一些莫名其妙的drop
all_data.drop(columns=['level_0','Unnamed: 0'],inplace=True)
#这个地方排序一下 后面就写的更舒服一些一定要注意 这里不排序 后面减就会有副职
all_data.sort_values(by='order_pay_time',inplace=True)

#下面查看缺失量
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head

#日期处理
all_data['order_pay_time'] = pd.to_datetime(all_data['order_pay_time'])
all_data['goods_list_time'] = pd.to_datetime(all_data['goods_list_time'])
all_data['goods_delist_time'] = pd.to_datetime(all_data['goods_delist_time'])
all_data['order_pay_date'] = all_data['order_pay_time'].dt.date


all_data['goods_list_date'] = all_data['goods_list_time'].dt.date
all_data['goods_delist_date'] = all_data['goods_delist_time'].dt.date
all_data['delta_time_list'] = all_data['goods_delist_date'] - all_data['goods_list_date'] 
all_data['delta_time_list'] = abs(all_data['delta_time_list'].dt.days)

all_data['7月-12月-商品上架']=all_data['goods_list_date'].astype(str).apply(lambda x: 1 if  '2013-07-03'<=x<= '2013-12-31' else 0)
all_data['1月-6月-商品上架']=all_data['goods_list_date'].astype(str).apply(lambda x: 1 if  '2013-01-01'<=x<= '2013-7-2' else 0)

##地区编码
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
all_data['new_location_code'] = label.fit_transform(all_data['new_location'])
print("日期处理完毕-地区编码也已经完成")

#这个时候先导入训练好的物品特征 
#通过制定聚类算法 已经打好了20种标签
#这里先不急着把他马上拼接上去
#而是要给每一个客人买过的id先打上标签让每一行都有商品的特征
#虽然无法解决提取到各个样本，后面可以取最大值 点评这个人最喜欢买什么

goods_feature=pd.read_csv('goods_feature.csv')
goods_feature.drop(columns='Unnamed: 0',inplace=True)
print("商品特征读取成功")

#下面要对商品种类开始处理
#请注意脑袋不要乱  这个地方不要乱merge
#因为我需要的是每一种类的值

#商品种类平均价格
Goods_kind__price_mean=goods_feature.groupby('Goods_clustering_kind')['goods_price_mean'].agg({'Goods_kind__price_mean':np.mean})
print(Goods_kind__price_mean,'\n')

#商品种类的会员折扣
Goods_kind_member_discount_mean=goods_feature.groupby('Goods_clustering_kind')['goods_memeber_discount'].agg({'Goods_member_discount_mean':np.mean})
print(Goods_kind_member_discount_mean,'\n')

#商品种类的在库状态
Goods_kind_status_mean=goods_feature.groupby('Goods_clustering_kind')['goods_status_mean'].agg({'Goods_kind_status_mean':np.mean})
print(Goods_kind_status_mean,'\n')

#商品种类的平均在架时间
Goods_kind_date_time_mean=goods_feature.groupby('Goods_clustering_kind')['deta_time_mean'].agg({'Goods_kind_date_time_mean':np.mean})
print(Goods_kind_date_time_mean,'\n')

#商品种类的最大优惠了多少
Goods_kind_discount=goods_feature.groupby('Goods_clustering_kind')['goods_discount'].agg({'Goods_kind_discount':np.mean})
print(Goods_kind_discount,'\n')

#商品种类的性别
Goods_kind_gender=goods_feature.groupby('Goods_clustering_kind')['goods_gender'].agg({'Goods_kind_gender':np.mean})

print(Goods_kind_gender,'\n')
all_data=pd.merge(all_data,goods_feature[['goods_id','Goods_clustering_kind']],on='goods_id')
print(all_data.shape)
print('所有的前期处理准备完毕')


(1400764, 33)
<bound method NDFrame.head of                      Missing Ratio
is_member_actived        60.622203
member_status            60.622203
customer_gender          60.622203
customer_city             6.104954
customer_province         6.104954
order_detail_amount       6.058622
is_customer_rate          6.058622
纬度                        0.000071
精度                        0.000071>
(1400763, 34)
日期处理完毕-地区编码也已经完成
(1400763, 37)
所有的前期处理准备完毕


In [10]:
#下面开始聚合用户特征

#第一步开始聚合商品
all_data['购买了商品+'] = list(map(lambda x: '已经购买的商品种类—' + str(x),all_data.Goods_clustering_kind))
temp = pd.crosstab(all_data['customer_id'],all_data['购买了商品+']).reset_index()
all_user=all_data[['customer_id']].drop_duplicates()
all_user=pd.merge(all_user,temp,on='customer_id')
all_user.head()

all_user=merge1(all_user)
print('已经把购买商品的属性加上去了')





购买了商品+,customer_id,已经购买的商品种类—0,已经购买的商品种类—1,已经购买的商品种类—10,已经购买的商品种类—11,已经购买的商品种类—12,已经购买的商品种类—13,已经购买的商品种类—14,已经购买的商品种类—15,已经购买的商品种类—16,已经购买的商品种类—17,已经购买的商品种类—18,已经购买的商品种类—19,已经购买的商品种类—2,已经购买的商品种类—3,已经购买的商品种类—4,已经购买的商品种类—5,已经购买的商品种类—6,已经购买的商品种类—7,已经购买的商品种类—8,已经购买的商品种类—9
0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,5.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6,6.0,6,0,1,1,1,2,1,1,0,1,0,1,2,0,1,0,1,0,1,2
7,7.0,0,0,0,0,0,0,11,0,0,0,0,0,0,1,0,0,0,0,0,0
8,8.0,0,0,0,0,0,0,11,0,0,0,0,0,0,1,0,0,0,0,0,0
9,9.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [67]:
all_customer=all_user

#用户总购买的商品数目五个属性
customer_buy_goods_num = all_data.groupby(['customer_id'], as_index=False)['order_total_num'].agg(
                                                                                  {  'order_total_num_max':'max',
                                                                                     'order_total_num_mean':'mean',
                                                                                     'order_total_num_min':'min',
                                                                                     'order_total_num_sum':'sum',
                                                                                     'order_total_num_median':'median'}
                                                                                                  )
print('ok')
#父订单商品总金额五个属性
customer_total_pay = all_data.groupby(['customer_id'], as_index=False)['order_amount'].agg(
                                                                                    {'order_amount_max':'max',
                                                                                     'order_amount_mean':'mean',
                                                                                     'order_amount_min':'min',
                                                                                     'order_amount_sum':'sum',
                                                                                     'order_amount_median':'median'})
print('ok')
#父订单实付金额
customer_real_pay = all_data.groupby(['customer_id'], as_index=False)['order_total_payment'].agg(
                                                                                    {'order_total_payment_max':'max',
                                                                                     'order_total_payment_mean':'mean',
                                                                                     'order_total_payment_min':'min',
                                                                                     'order_total_payment_sum':'sum',
                                                                                     'order_total_payment_median':'median'})
print('ok')
#父订单优惠金额
customer_happy_pay = all_data.groupby(['customer_id'], as_index=False)['order_total_discount'].agg(
                                                                                    {'order_total_discount_max':'max',
                                                                                     'order_total_discount_mean':'mean',
                                                                                     'order_total_discount_min':'min',
                                                                                     'order_total_discount_sum':'sum',
                                                                                     'order_total_discount_median':'median'})
print('ok')
#父订单包含的子订单数量的五个属性
customer_number = all_data.groupby(['customer_id'], as_index=False)['order_count'].agg(
                                                                                    {'order_count_max':'max',
                                                                                     'order_count_mean':'mean',
                                                                                     'order_count_min':'min',
                                                                                     'order_count_sum':'sum',
                                                                                     'order_count_median':'median'})
print('ok')
#子订单实付金额
customer_son_real_pay = all_data.groupby(['customer_id'], as_index=False)['order_detail_payment'].agg(
                                                                                    {'order_detail_payment_max':'max',
                                                                                     'order_detail_payment_mean':'mean',
                                                                                     'order_detail_payment_min':'min',
                                                                                     'order_detail_payment_sum':'sum',
                                                                                     'order_detail_payment_median':'median'})
#子订单优惠金额
print('ok')
customer_son_happy_pay = all_data.groupby(['customer_id'], as_index=False)['order_detail_discount'].agg(
                                                                                    {'order_detail_discount_max':'max',
                                                                                     'order_detail_discount_mean':'mean',
                                                                                     'order_detail_discount_min':'min',
                                                                                     'order_detail_discount_sum':'sum',
                                                                                     'order_detail_discount_median':'median'})
print('ok')
#购买时间间隔 这个地方不要提取错了
q=all_data.groupby(['customer_id'], as_index=False)['order_pay_time'].agg({'deta_time_mean':(lambda x:x.diff().mean()),
                                                                           'deta_time_max':(lambda x:x.diff().max()),
                                                                           'deta_time_min':(lambda x:x.diff().min())
                                                                          })
#要是中间就买了一次 我们认为时间间隔为0
q.fillna(0,inplace=True)
q['deta_time_mean']=q['deta_time_mean'].dt.days
q['deta_time_max']=q['deta_time_max'].dt.days
q['deta_time_min']=q['deta_time_min'].dt.days
print('暂时就想到这么多人的特征')

all_customer=pd.merge(all_customer,customer_buy_goods_num,on='customer_id')
all_customer=pd.merge(all_customer,customer_total_pay,on='customer_id')
all_customer=pd.merge(all_customer,customer_real_pay,on='customer_id')
all_customer=pd.merge(all_customer,customer_happy_pay,on='customer_id')
all_customer=pd.merge(all_customer,customer_number,on='customer_id')
all_customer=pd.merge(all_customer,customer_son_real_pay,on='customer_id')
all_customer=pd.merge(all_customer,customer_son_happy_pay,on='customer_id')
all_customer.shape

ok
ok
ok
ok
ok
ok
ok
暂时就想到这么多人的特征


(929806, 176)

In [68]:
all_customer.columns

Index(['customer_id', '已经购买的商品种类—0', '已经购买的商品种类—1', '已经购买的商品种类—10',
       '已经购买的商品种类—11', '已经购买的商品种类—12', '已经购买的商品种类—13', '已经购买的商品种类—14',
       '已经购买的商品种类—15', '已经购买的商品种类—16',
       ...
       'order_detail_payment_max', 'order_detail_payment_mean',
       'order_detail_payment_min', 'order_detail_payment_sum',
       'order_detail_payment_median', 'order_detail_discount_max',
       'order_detail_discount_mean', 'order_detail_discount_min',
       'order_detail_discount_sum', 'order_detail_discount_median'],
      dtype='object', length=176)

In [74]:
all_data['new_location_code']

0           49
1          302
2          179
3          219
4          178
5          281
6           80
7          226
8          248
9          112
10           0
11          78
12          49
13          90
14         194
15          29
16         219
17         124
18          74
19          69
20         349
21         349
22         143
23           0
24         275
25          29
26           0
27          29
28           0
29         198
          ... 
1400733    219
1400734    349
1400735    230
1400736    233
1400737    199
1400738    171
1400739    206
1400740      2
1400741    273
1400742    174
1400743    287
1400744      1
1400745    168
1400746    227
1400747     49
1400748     69
1400749    124
1400750    202
1400751     29
1400752    248
1400753    116
1400754    225
1400755    308
1400756     29
1400757    319
1400758    124
1400759    179
1400760    213
1400761    183
1400762      0
Name: new_location_code, Length: 1400763, dtype: int32

In [73]:
new_location_code

NameError: name 'new_location_code' is not defined

In [ ]:
all_data=pd.read_csv('D:/new_date.csv',low_memory=False)
all_data1 = all_data.sample(frac=0.01)
import numpy as np

all_data1['order_pay_time'] = pd.to_datetime(all_data1['order_pay_time'])
all_data1['order_pay_date'] = all_data1['order_pay_time'].dt.date
all_data1.sort_values(by='order_pay_time',inplace=True)
# goods_buy_deltatime = all_data1.groupby(['customer_id'], as_index=False)['order_pay_time'].agg({'deta_time_max':lambda x:1 if np.isnan(x.diff().max().days) 
#                                                                                                 else x.diff().max().days })
#用户购买间隔
# goods_buy_deltatime = all_data1.groupby(['customer_id'], as_index=False)['order_pay_date'].agg({'deta_time_mean':lambda x:1 if np.isnan(x.diff().mean().dt.days) 
#                                                                                                        else 0
                                                                                                #x.diff().mean().dt.days
#                                                                                        'deta_time_min':lambda x:1 if np.isnan(x.diff().min()) 
#                                                                                                        else x.diff().min().days,
#                                                                                        'deta_time_max':lambda x:1 if np.isnan(x.diff().max()) 
#                                                                                                        else x.diff().max().days,
#                                                                                        'deta_time_median':lambda x:1 if np.isnan(x.diff().median()) 
#                                                                                                        else x.diff().median().days
  #                                                                                     })

In [ ]:
q=all_data1.groupby(['customer_id'], as_index=False)['order_pay_time'].agg({'deta_time_mean':(lambda x:x.diff().mean()),
                                                                           'deta_time_max':(lambda x:x.diff().max()),
                                                                           'deta_time_min':(lambda x:x.diff().min())
                                                                          })
#要是中间就买了一次 我们认为时间间隔为0
q.fillna(0,inplace=True)
q['deta_time_mean']=q['deta_time_mean'].dt.days
q['deta_time_max']=q['deta_time_max'].dt.days
q['deta_time_min']=q['deta_time_min'].dt.days
q.info()

In [ ]:
q.info()

In [ ]:
all_data1['购买次数']=1
all_data1.groupby('customer_id')['购买次数'].count().sort_values(ascending=False)

In [ ]:
all_data1[all_data1['customer_id']==417999.0]

In [ ]:
q['deta_time_mean'].apply(lambda x:1 if np.isnan(x.days) else x.days)

In [ ]:
all_data1[all_data1['customer_id']==257]

In [ ]:
np.isnan(q['deta_time_mean'].dt.days)

In [ ]:
pd.set_option('display.max_columns', None)
all_data1.head()

In [ ]:
import pandas as pd

In [ ]:
example=pd.DataFrame({'交易时间':[14,24,2,2,5,6,6,3]
                     ,'人名':['小吴','小王','阿黑','小吴','小王','阿黑','小吴','小王']})
example

In [ ]:
goods_buy_deltatime=example.groupby(['人名'],as_index=False)['交易时间'].diff()
#.reset_index()
#.agg({'cuttime':'diff'})
goods_buy_deltatime

In [ ]:
goods_buy_deltatime
apply
#.apply(lambda x: x.dt.date ) 

#.apply(lambda x:x.dt.days.diff())

In [ ]:
q.values[0][1].days

In [ ]:
ss=goods_buy_deltatime['deta_time_mean'].sort_values()

In [ ]:

ss.astype('float')

In [ ]:
all_data1[all_data1['customer_id']==209.0]

In [ ]:
df.groupby('B').apply()

In [ ]:
all_data['购买次数']=1
all_data.groupby(['customer_id'], as_index=False)['购买次数'].count().sort_values(by='购买次数',ascending=False)
import numpy as np
#用户购买间隔
# #client_buy_deltatime = 
# for i,j in all_data.groupby(['customer_id'], as_index=False)['order_pay_time']:
#     print(i)
#     print(j)
#     break
# .agg(
#     {
#     'deta_time_min':lambda x:x.diff().min().days,
#     })
#.sort_values()
def check(df):
    return df.sort_values()
goods_buy_deltatime = all_data.groupby(['customer_id'], as_index=False)['order_pay_time'].apply(check)
                                                                                    #{'deta_time_max':x.diff().dt.days.max()}

In [ ]:
all_data['order_pay_time'] = pd.to_datetime(all_data['order_pay_time'])
all_data['goods_list_time'] = pd.to_datetime(all_data['goods_list_time'])
all_data['goods_delist_time'] = pd.to_datetime(all_data['goods_delist_time'])
all_data['order_pay_date'] = all_data['order_pay_time'].dt.date
q=all_data[all_data['customer_id']==15182]['order_pay_date']

In [ ]:
q

In [ ]:
.diff().dt.days.values